# Set environment

In [2]:
library(tidyverse)
library(IRdisplay)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [98]:
source("./data/beetle.R")
beetle

dose,n,y,n_y
1.6907,59,6,53
1.7242,60,13,47
1.7552,62,18,44
1.7842,56,28,28
1.8113,63,52,11
1.8369,59,53,6
1.8610,62,61,1
1.8839,60,60,0


In [4]:
fit_logit   = glm(cbind(y, n-y) ~ dose, data = beetle, binomial(link = "logit"))
fit_cloglog = glm(cbind(y, n-y) ~ dose, data = beetle, binomial(link = "cloglog"))

In [5]:
fit_logit$coef

(Intercept)        dose 
  -60.71745    34.27033

In [6]:
fit_cloglog$coef

(Intercept)        dose 
  -39.57231    22.04117

- pattern 1 dose = 1.8
- pattern 2 dose = 1.81

In [7]:
pattern1 = c(1, 1.80)
pattern2 = c(1, 1.81)

# link: logit

$g(p) = logit(p) = log(\frac{p}{1-p}) = log(odds)$

OR_<sub>pattern 2 vs pattern 1</sub>

$= \frac{
    odds (pattern 2)
}{
    odds (pattern 1)
}$

$= \frac{
    exp \big( \beta_1 + \beta_2 * 1.81 \big)
}{
    exp \big( \beta_1 + \beta_2 * 1.80 \big)
}$

$= exp \big( \beta_2 * 0.01 \big)$

$$\eta = \beta_1 + \beta_2 * x$$

In [8]:
eta1 = pattern1 %*% fit_logit$coef %>% as.vector
display(eta1)

[1] 0.9691318

In [9]:
eta2 = pattern2 %*% fit_logit$coef %>% as.vector
display(eta2)

[1] 1.311835

$$g(p) = logit(p) = log(\frac{p}{1-p})$$

$$p = g^{-1}(\eta) = \frac{1}{1 + exp(\eta)}$$

In [10]:
g_inv = function(eta){
    1 / (1 + exp(-eta))
} # end func

In [11]:
p1 = g_inv(eta1)
display(p1)

[1] 0.7249464

In [12]:
p2 = g_inv(eta2)
display(p2)

[1] 0.7878201

$$odds = \frac{p}{1-p}$$

In [13]:
odds1 = p1 / (1 - p1)
display(odds1)

[1] 2.635655

In [14]:
odds2 = p2 / (1 - p2)
display(odds2)

[1] 3.712981

$$OR_{\text{pattern 2 vs pattern 1}} = \frac{
    odds (\text{pattern 2})
}{
    odds (\text{pattern 1})
}$$

In [15]:
odds2 / odds1

[1] 1.408751

since the regression use logit link, we can calculate the odds ratio from $\beta$ directly

In [16]:
exp(fit_logit$coefficients[2] * (0.01))

dose 
1.408751

# link: cloglog

$$\eta = \beta_1 + \beta_2 * x$$

In [17]:
eta1 = pattern1 %*% fit_cloglog$coef %>% as.vector
display(eta1)

[1] 0.1017951

In [18]:
eta2 = pattern2 %*% fit_cloglog$coef %>% as.vector
display(eta2)

[1] 0.3222068

$$g(p) = log[-log(1-p)] = \eta$$

$$p = g^{-1}(\eta) = 1 - exp(-exp(\eta))$$


In [19]:
g_inv = function(eta){
    1 - exp(-exp(eta))
} # end func

In [20]:
p1 = g_inv(eta1)
display(p1)

[1] 0.6695026

In [21]:
p2 = g_inv(eta2)
display(p2)

[1] 0.7484642

$$odds = \frac{p}{1-p}$$

In [22]:
odds1 = p1 / (1 - p1)
display(odds1)

[1] 2.025743

In [23]:
odds2 = p2 / (1 - p2)
display(odds2)

[1] 2.975578

$$OR_{\text{pattern 2 vs pattern 1}} = \frac{
    odds (\text{pattern 2})
}{
    odds (\text{pattern 1})
}$$

In [24]:
odds2 / odds1

[1] 1.468882

# Compare link logit and cloglog

In [25]:
fit_logit   = glm(cbind(y, n-y) ~ dose, data = beetle, binomial(link = "logit"))
fit_cloglog = glm(cbind(y, n-y) ~ dose, data = beetle, binomial(link = "cloglog"))

In [26]:
g_inv_logit = function(eta){
    1 / (1 + exp(-eta))
} # end func

g_inv_cloglog = function(eta){
    1 - exp(-exp(eta))
} # end func

In [27]:
get_odds = function(p){
    p / (1-p)
}

In [28]:
mat = matrix(c(NA, NA, NA, NA), 2, 2)
rownames(mat) = c("1.81 vs 1.80", "1.71 vs 1.70")
colnames(mat) = c("logit", "cloglog")
mat

,logit,cloglog
1.81 vs 1.80,NA,NA
1.71 vs 1.70,NA,NA


In [29]:
pattern1 = c(1, 1.80)
pattern2 = c(1, 1.81)

odds1 = pattern1 %*% fit_logit$coef %>% g_inv_logit %>% get_odds
odds2 = pattern2 %*% fit_logit$coef %>% g_inv_logit %>% get_odds
mat["1.81 vs 1.80", "logit"] = odds2 / odds1

odds1 = pattern1 %*% fit_cloglog$coef %>% g_inv_cloglog %>% get_odds
odds2 = pattern2 %*% fit_cloglog$coef %>% g_inv_cloglog %>% get_odds
mat["1.81 vs 1.80", "cloglog"] = odds2 / odds1

pattern1 = c(1, 1.70)
pattern2 = c(1, 1.71)

odds1 = pattern1 %*% fit_logit$coef %>% g_inv_logit %>% get_odds
odds2 = pattern2 %*% fit_logit$coef %>% g_inv_logit %>% get_odds
mat["1.71 vs 1.70", "logit"] = odds2 / odds1

odds1 = pattern1 %*% fit_cloglog$coef %>% g_inv_cloglog %>% get_odds
odds2 = pattern2 %*% fit_cloglog$coef %>% g_inv_cloglog %>% get_odds
mat["1.71 vs 1.70", "cloglog"] = odds2 / odds1

display(mat)

,logit,cloglog
1.81 vs 1.80,1.408751,1.468882
1.71 vs 1.70,1.408751,1.265946


observation:

In logit model, the **OR associated with 0.01 unit increase in dose** <font color = "red">does not depend</font> on dose

In cloglog model, the **OR associated with 0.01 unit increase in dose** <font color = "red">depend</font> on dose

The question is: <font color = "red">what link function to choose?</font> For logit and cloglog related link functions, there is a way to test which link function is better. 

Before introducing the test, we need to introduce the **general family of link functions**

# General family of link functions

[Aranda-Ordaz F (1981). "On two families of transformations to additivity for binary response data"](https://academic.oup.com/biomet/article-abstract/68/2/357/260360?redirectedFrom=PDF)

$$g(p_i; \alpha) = log \Big\{ 
    \frac{
        (1 - p_i)^{-\alpha} - 1
    }{
        \alpha
    }
\Big\}$$

<font color = "blue">$\alpha = 1$</font>

$g(p_i; \alpha) = log \Big\{ 
    \frac{
        (1 - p_i)^{-1} - 1
    }{
        1
    }
\Big\} = log \Big\{ \frac{p_i}{1 - p_i} \Big\}$

<font color = "blue">$\alpha \rightarrow 0$</font>

$g(p_i; \alpha) \rightarrow log[-log(1-p)]$

**1st order approximation of Taylor expansion**

Suppose the true $\alpha$ is $\alpha_0$

$$g(p_i ; \alpha) \sim g(p_i ; \alpha_0) + (\alpha - \alpha_0) \frac{\partial g(p_i ; \alpha)}{\partial \alpha} \Big|_{\alpha_0}$$

$$g(p_i ; \alpha) = \eta_i \sim g(p_i ; \alpha_0) - \gamma z_i$$ where $$\gamma = \alpha_0 - \alpha ; z_i = \frac{\partial g(p_i ; \alpha)}{\partial \alpha} \Big|_{\alpha_0}$$

After rearrange
$$g(p_i ; \alpha_0) \sim \eta_i + \gamma z_i$$

recall that 
$$z_i = \frac{\partial g(p_i ; \alpha)}{\partial \alpha} \Big|_{\alpha_0}$$

Where we could derive 

$\frac{\partial g(p_i ; \alpha)}{\partial \alpha}$

$= \frac{\log{(1 - p_i)}}{(1 - p_i)^\alpha - 1} - \alpha^{-1}$

In [30]:
dG = function(p, alpha){
    log(1 - p) / ((1 - p)^alpha - 1) - alpha^{-1}
} # end func

## original model $\alpha_0 = 1$

In [31]:
fit_logit = glm(cbind(y, n - y) ~ dose, data = beetle, family = binomial(link = "logit"))

## fit a glm model to test current $\alpha$

calculate $z_i$ using $\hat{p_i}$ and $\alpha$

In [35]:
tmp   = beetle
tmp$z = dG(fit_logit$fitted.values, 1)
display(tmp)

dose,n,y,n_y,z
1.6907,59,6,53,0.03049800
1.7242,60,13,47,0.09225343
1.7552,62,18,44,0.24159057
1.7842,56,28,28,0.53584036
1.8113,63,52,11,0.99401390
1.8369,59,53,6,1.58567955
1.8610,62,61,1,2.25112322
1.8839,60,60,0,2.94830490


fit into regression model to get $\gamma$

$g(p_i ; \alpha_0) \sim \eta_i + \gamma z_i$

In [36]:
fit_link = glm(cbind(y, n - y) ~ dose + z, data = tmp, family = binomial(link = "logit"))
fit_link


Call:  glm(formula = cbind(y, n - y) ~ dose + z, family = binomial(link = "logit"), 
    data = tmp)

Coefficients:
(Intercept)         dose            z  
    -25.956       14.105        1.664  

Degrees of Freedom: 7 Total (i.e. Null);  5 Residual
Null Deviance:	    284.2 
Residual Deviance: 2.953 	AIC: 35.15

In [42]:
gamma = fit_link$coef["z"]
display(gamma)

z 
1.663775

## test whether the parameter $\gamma$ is significant or not 

if <font color = "blue">not significant</font>, we could not reject $\gamma = 0$; that is, not rejecting $\alpha = \alpha_0$

if <font color = "red">significant</font>, we reject $\gamma = 0$; that is, decide $\alpha \neq \alpha_0$

In [38]:
anova(fit_link, test = "Chisq")

,Df,Deviance,Resid. Df,Resid. Dev,Pr(>Chi)
NULL,NA,NA,7,284.202449,NA
dose,1,272.970218,6,11.232231,2.556089e-61
z,1,8.279424,5,2.952807,4.009684e-03


## Since it is significant, we need to calculate possible $\alpha$

current $\alpha_0 = 1$ and $\gamma = \alpha_0 - \alpha$

Therefore, a new $\hat{\alpha} =  \alpha_0 - \gamma$

In [43]:
alpha_hat = 1 - gamma
display(alpha_hat)

z 
-0.6637749

Since $0 < \alpha \leq 1$, we could suggest a better $\hat{\alpha} = 0$; that is, we will fit a **cloglog** link

In [44]:
fit_logit   = glm(cbind(y, n-y) ~ dose, data = beetle, binomial(link = "logit"))
fit_cloglog = glm(cbind(y, n-y) ~ dose, data = beetle, binomial(link = "cloglog"))

### compare two models

predicted values

In [86]:
mat = cbind(beetle$y, beetle$n * fit_logit$fitted.values, beetle$n * fit_cloglog$fitted.values)
colnames(mat) = c("y", "logit", "cloglog")
display(mat)

,y,logit,cloglog
1,6,3.457461,5.58945
2,13,9.841672,11.28068
3,18,22.451378,20.95422
4,28,33.897635,30.36944
5,52,50.095822,47.77642
6,53,53.290913,54.14273
7,61,59.222159,61.11331
8,60,58.742961,59.94723


model comparison

In [80]:
mat = rbind(
    ### deviance
    c(fit_logit$deviance, 
      fit_cloglog$deviance),
    
    ### Pearson X2
    c(sum(residuals(fit_logit,   type = "pearson")^2),
      sum(residuals(fit_cloglog, type = "pearson")^2)),
    
    ### AIC
    c(fit_logit$aic,
      fit_cloglog$aic)
) # end cbind

rownames(mat) = c("Deviance", "Pearson X2", "AIC")
colnames(mat) = c("logit", "cloglog")

display(mat)

,logit,cloglog
Deviance,11.23223,3.446439
Pearson X2,10.02682,3.294694
AIC,41.43027,33.644477


Since they are not nested model, it is not exactly correct to compare using deviance. But expected value of deviance is the degree of freedom (n - p = 8 - 2 = 6). 

Another index to compare is AIC

### Appendix: residuals of chi-square
$$\frac{y_i - n_i \pi_i}{\sqrt{n_i \pi_i (1 - \pi_i)}}$$

calculate manually

In [73]:
(beetle$y - beetle$n * fit_logit$fitted.values) / (beetle$n * fit_logit$fitted.values * (1 - fit_logit$fitted.values))^0.5

1          2          3          4          5          6          7 
 1.4092960  1.1011003 -1.1762596 -1.6123815  0.5944454 -0.1281090  1.0914228 
         8 
 1.1331102

use function residuals with type = "pearson"

In [69]:
residuals(fit_logit, type = "pearson")

1          2          3          4          5          6          7 
 1.4092960  1.1011003 -1.1762596 -1.6123815  0.5944454 -0.1281090  1.0914228 
         8 
 1.1331102

calcualte pearson chi-square

In [89]:
sum(residuals(fit_logit,   type = "pearson")^2)

[1] 10.02682

### residuals

In [94]:
fit_logit$residuals

1           2           3           4           5           6 
 0.78115418  0.38388091 -0.31082206 -0.44081641  0.18557365 -0.05641516 
          7           8 
 0.67002811  1.02139898

### deviance residuals

In [93]:
beetle$y / beetle$n - fit_logit$fitted.values

1            2            3            4            5            6 
 0.043093890  0.052638798 -0.071796425 -0.105314906  0.030225053 -0.004930735 
           7            8 
 0.028674861  0.020950656

In [92]:
beetle$y - beetle$n * fit_logit$fitted.values

1          2          3          4          5          6          7 
 2.5425395  3.1583279 -4.4513784 -5.8976348  1.9041784 -0.2909133  1.7778414 
         8 
 1.2570394

In [90]:
residuals(fit_logit, type = "deviance")

1          2          3          4          5          6          7 
 1.2836777  1.0596900 -1.1961123 -1.5941244  0.6061405 -0.1271584  1.2510711 
         8 
 1.5939850

In [96]:
sum(residuals(fit_logit, type = "deviance"))

[1] 2.877169

In [100]:
beetle

dose,n,y,n_y
1.6907,59,6,53
1.7242,60,13,47
1.7552,62,18,44
1.7842,56,28,28
1.8113,63,52,11
1.8369,59,53,6
1.8610,62,61,1
1.8839,60,60,0


In [108]:
tmp = beetle_n1 #%>% unique
dim(tmp)

[1] 481   3

In [107]:
fit_tmp = glm(cbind(y, n- y)~dose, data = beetle_n1, family = binomial(link = "logit"))
fit_tmp$fitted

1          2          3          4          5          6          7 
0.05860103 0.05860103 0.05860103 0.05860103 0.05860103 0.05860103 0.16402787 
         8          9         10         11         12         13         14 
0.16402787 0.16402787 0.16402787 0.16402787 0.16402787 0.16402787 0.16402787 
        15         16         17         18         19         20         21 
0.16402787 0.16402787 0.16402787 0.16402787 0.16402787 0.36211901 0.36211901 
        22         23         24         25         26         27         28 
0.36211901 0.36211901 0.36211901 0.36211901 0.36211901 0.36211901 0.36211901 
        29         30         31         32         33         34         35 
0.36211901 0.36211901 0.36211901 0.36211901 0.36211901 0.36211901 0.36211901 
        36         37         38         39         40         41         42 
0.36211901 0.36211901 0.60531491 0.60531491 0.60531491 0.60531491 0.60531491 
        43         44         45         46         47         48         49 
0.60531491 0.60531491 0.60531491 0.60531491 0.60531491 0.60531491 0.60531491 
        50         51         52         53         54         55         56 
0.60531491 0.60531491 0.60531491 0.60531491 0.60531491 0.60531491 0.60531491 
        57         58         59         60         61         62         63 
0.60531491 0.60531491 0.60531491 0.60531491 0.60531491 0.60531491 0.60531491 
        64         65         66         67         68         69         70 
0.60531491 0.60531491 0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 
        71         72         73         74         75         76         77 
0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 
        78         79         80         81         82         83         84 
0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 
        85         86         87         88         89         90         91 
0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 
        92         93         94         95         96         97         98 
0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 
        99        100        101        102        103        104        105 
0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 
       106        107        108        109        110        111        112 
0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 
       113        114        115        116        117        118        119 
0.79517177 0.79517177 0.79517177 0.79517177 0.79517177 0.90323582 0.90323582 
       120        121        122        123        124        125        126 
0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 
       127        128        129        130        131        132        133 
0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 
       134        135        136        137        138        139        140 
0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 
       141        142        143        144        145        146        147 
0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 
       148        149        150        151        152        153        154 
0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 
       155        156        157        158        159        160        161 
0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 
       162        163        164        165        166        167        168 
0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 0.90323582 
       169        170        171        172        173        174        175 
0.90323582 0.90323582 0.95519611 0.95519611 0.95519611 0.95519611 0.95519611 
       176        177        178        179        180        181        182 
0.95519611 0.95519611 0.9551961

In [99]:
beetle_n1

dose,n,y
1.6907,1,1
1.6907,1,1
1.6907,1,1
1.6907,1,1
1.6907,1,1
1.6907,1,1
1.7242,1,1
1.7242,1,1
1.7242,1,1
1.7242,1,1


In [106]:
?glm